## Setup

### Reduce Tahoe region greenhouse gas (GHG) emissions and support the measurement of carbon sequestration.  

### Track poor air quality, wildfire smoke, and extreme heat trends regionally.

### Lake Tahoe water level

### Annual average water temperature, including surface water temperature

### Lake clarity measured by Secchi Depth

### Total precipitation in water per year, extreme precipitation, and snow as a fraction of annual precipitation

### Acres of forest fuels reduction treated for wildfire in high-risk areas, map of prescribed fire treatment projects

### Tree species diversity and increasing old growth forest 

### Probability of fire by low, moderate, & high severity by management zone. *We are expecting additional clarity regarding this indicator.  

### Acres treated for aquatic invasive species. *We could gain additional clarity regarding this indicator.

### Acres of restored high-quality wetlands and meadows helping to store flood waters. *We are expecting additional clarity regarding this indicator.  

### Increased number of parcels with Stormwater Best Management Practices (BMPs) improvements

### Increase in square feet of urban development treated by areawide stormwater infrastructure within key watersheds. *We are expecting additional clarity regarding this indicator.  

### Total number of housing units in town centers and share of affordable housing in Town Centers

In [ ]:
towncenterService               = "https://maps.trpa.org/server/rest/services/LocalPlan/MapServer/2"
parcelDevelopmentHistoryService = "https://maps.trpa.org/server/rest/services/"

### Change in share of homes with electric or solar energy fuel compared to oil/gas over time

### Number of deed-restricted affordable, moderate, and achievable units

In [95]:
import pandas as pd
import numpy as np
import plotly.express as px

# deed restriction service
deedRestrictionService = "https://www.laketahoeinfo.org/WebServices/GetDeedRestrictedParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476"

# read in deed restricted parcels
dfDeed = pd.read_json(deedRestrictionService)

# filter out deed restrictions that are not affordable housing
dfDeed = dfDeed.loc[dfDeed['DeedRestrictionType'].isin(['Affordable Housing', 'Achievable Housing', 'Moderate Income Housing'])]

# create year column
dfDeed['Year'] = dfDeed['RecordingDate'].str[-4:]

# group by type and year
df = dfDeed.groupby(['DeedRestrictionType', 'Year']).size().reset_index(name='Total')

# sort by year
df.sort_values('Year', inplace=True)

# rename columns
df = df.rename(columns={'DeedRestrictionType': 'Type', 'Year': 'Year', 'Total': 'Count', 'CumulativeTotal': 'Cumulative Count'})

# Create a DataFrame with all possible combinations of 'Type' and 'Year'
df_all = pd.DataFrame({
    'Type': np.repeat(df['Type'].unique(), df['Year'].nunique()),
    'Year': df['Year'].unique().tolist() * df['Type'].nunique()
})

# Merge the new DataFrame with the original one to fill in the gaps of years for each type with NaN values
df = pd.merge(df_all, df, on=['Type', 'Year'], how='left')

# Replace NaN values in 'Count' with 0
df['Count'] = df['Count'].fillna(0)

# Ensure 'Count' is of integer type
df['Count'] = df['Count'].astype(int)

# Recalculate 'Cumulative Count' as the cumulative sum of 'Count' within each 'Type' and 'Year'
df['Cumulative Count'] = df.sort_values('Year').groupby('Type')['Count'].cumsum()

# create cumuluative total of deed restricted parcels by type
fig = px.line(df, x="Year", y="Cumulative Count", color="Type", title="Deed Restricted Parcels")
fig.show()


### Percent of renewable energy as a share of total energy used

### Total transit ridership by transit systems
* https://www.laketahoeinfo.org/Indicator/Detail/46/Overview
* this doesn't work yet. Talking to ESA to get web service stood up.

In [2]:
# get EIP indicator as dataframe
import pandas as pd
import plotly.express as px

# indicator #46
eipTransit = "https://www.laketahoeinfo.org/WebServices/GetReportedEIPIndicatorProjectAccomplishments/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476/46"

# read in as dataframe

""


### Daily per capita Vehicles Miles Traveled (VMT) and progress towards VMT target. The RTP (2021) identifies a Daily per capita VMT target set at a 6.8 percent reduction from 2018 levels by 2045 (2018 per capita daily VMT is 12.48, goal is 11.63).

In [ ]:
## new data from Josh/RSG?

### Coverage of electric bus routes and alternative fuel, such as EV charging for vehicles and bicycles
* Primary source: https://www.plugshare.com/ requested access to their API
* Secondary (if needed to cross-check): https://afdc.energy.gov/stations/#/find/nearest

* current in-house data: https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/0

### Baseline mode share and weekday or seasonal variation. The Tahoe RTP (2021) includes the following Non-Auto Mode Share Target: Improve average non-auto mode share calculated from the two most recent TRPA travel survey results; the current performance on target at 24.5% (2018-20 average) up from 18% in 2014-16.

### Transportation access in priority communities. The Tahoe RTP (2021) includes a target to increase access to each mode for Priority communities to 100% by 2014.

### Increased lane miles of low-stress bicycle facilities (both bicycle and pedestrian facilities that are considered comfortable enough for all users and abilities, and implicitly measures active transportation network connectivity)

In [ ]:

# group by type and year
df = dfDeed.groupby(['DeedRestrictionType', 'Year']).size().reset_index(name='Total')

# sort by year
df.sort_values('Year', inplace=True)

# rename columns
df = df.rename(columns={'DeedRestrictionType': 'Type', 'Year': 'Year', 'Total': 'Count', 'CumulativeTotal': 'Cumulative Count'})

# Create a DataFrame with all possible combinations of 'Type' and 'Year'
df_all = pd.DataFrame({
    'Type': np.repeat(df['Type'].unique(), df['Year'].nunique()),
    'Year': df['Year'].unique().tolist() * df['Type'].nunique()
})

# Merge the new DataFrame with the original one to fill in the gaps of years for each type with NaN values
df = pd.merge(df_all, df, on=['Type', 'Year'], how='left')

# Replace NaN values in 'Count' with 0
df['Count'] = df['Count'].fillna(0)

# Ensure 'Count' is of integer type
df['Count'] = df['Count'].astype(int)

# Recalculate 'Cumulative Count' as the cumulative sum of 'Count' within each 'Type' and 'Year'
df['Cumulative Count'] = df.sort_values('Year').groupby('Type')['Count'].cumsum()

# create cumuluative total of deed restricted parcels by type
fig = px.line(df, x="Year", y="Cumulative Count", color="Type", title="Deed Restricted Parcels")
fig.show()

In [100]:
sdf_bikelane

,OBJECTID,CLASS,NAME,MILES,FROM_,TO_,WNTR_MAINT,MAINT_JURS,YR_OF_CONS,IMPLEMENT,COUNTY,E_BIKE,GlobalID,created_user,created_date,last_edited_user,last_edited_date,Shape.STLength(),SHAPE
0,1036,1,VILLAGE BLVD (NORTH),0.546032,ACE COURT,NORTHWOOD BLVD,YES,WASHOE COUNTY,Before 2006,WASHOE COUNTY,WASHOE COUNTY,,{5ABEA349-F0A3-4425-A7D2-E436B94B0B95},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,878.754949,"{""paths"": [[[763010.0494420798, 4349230.453223..."
1,1037,1,NORTHWOOD BLVD,0.353422,VILLAGE BLVD (NORTH),NORTHWOOD MIDBLOCK CROSSING,YES,WASHOE COUNTY,Before 2006,WASHOE COUNTY,WASHOE COUNTY,,{584CE2F1-8DC2-4ABB-8536-0DAFF06D8EFC},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,568.778879,"{""paths"": [[[763010.0494420798, 4349230.453223..."
2,1038,1,NORTHWOOD BLVD,0.264228,NORTHWOOD MIDBLOCK CROSSING,STATE ROUTE 28,YES,WASHOE COUNTY,Before 2006,WASHOE COUNTY,WASHOE COUNTY,,{8D883CF4-928B-4442-8F22-6BCD62F14931},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,425.235283,"{""paths"": [[[762435.8466366101, 4349243.555121..."
3,1039,1,VILLAGE BLVD (NORTH),0.190352,NORTHWOOD BLVD,STATE ROUTE 28,YES,WASHOE COUNTY,Before 2006,WASHOE COUNTY,WASHOE COUNTY,,{494DD914-74B2-4F2A-A530-09E9D09BFA10},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,306.341967,"{""paths"": [[[763010.0494420798, 4349230.453223..."
4,1040,1,MAYS BLVD,0.333108,LAKESHORE BLVD,ALLEN WAY,YES,WASHOE COUNTY,Before 2006,WASHOE COUNTY,WASHOE COUNTY,,{C5B9C56B-E9C5-41C1-82FA-5F5D3EA5FD8E},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,536.08629,"{""paths"": [[[762034.7960026348, 4348607.335126..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,1242,PED,SR 89 SIDEWALKS (EAST SIDE),<NA>,SOUTH TAHOE Y,15TH STREET,NO,CITY OF SOUTH LAKE TAHOE,2016,CITY OF SOUTH LAKE TAHOE,CITY OF SOUTH LAKE TAHOE,<NA>,{F36AD7CD-D9CC-44D9-A5E8-8F560F74D0B5},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,1863.130414,"{""paths"": [[[759290.486654425, 4311832.7693805..."
207,1243,3,<NA>,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{1790DF53-A7B2-4C03-A16E-4187FBBD5C60},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,620.525295,"{""paths"": [[[757739.6073097438, 4305006.863504..."
208,1244,3,<NA>,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,{7F79D2C2-ECCF-495F-8D1D-09E1CD04D16B},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,402.991628,"{""paths"": [[[758561.4390744474, 4305341.446249..."
209,1245,1,Jameson Beach Rd Path,<NA>,Hwy 50,Beacon,No,USFS,2022,USFS,El Dorado County,No,{975474C1-0637-4080-9F7F-E3937821DC95},MBINDL,2024-01-24 20:41:45,MBINDL,2024-01-24 20:41:45,366.811749,"{""paths"": [[[756561.7185036577, 4313677.252976..."


In [102]:
from arcgis.features import FeatureLayer
import pandas as pd

# Define the service URL
bikelaneService = "https://maps.trpa.org/server/rest/services/Transportation/MapServer/3"

# Create a FeatureLayer object
feature_layer = FeatureLayer(bikelaneService)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_bikelane = feature_layer.query().sdf

# Convert the series object to a dataframe
df_bikelane = pd.DataFrame.from_records(sdf_bikelane)

df_bikelane.Miles = df_bikelane['SHAPE'].length / 1609.34

df_bikelane



AttributeError: 'Series' object has no attribute 'length'

In [90]:
from arcgis.features import FeatureLayer

# Define the service URL
bikelaneService = "https://maps.trpa.org/server/rest/services/Transportation/MapServer/3"

# Create a FeatureLayer object
feature_layer = FeatureLayer(bikelaneService)

# Query the feature layer and convert the result to a Spatially Enabled DataFrame
sdf_bikelane = feature_layer.query().sdf

# filter for CLAss = 1
filtered_sdf_bikelane = sdf_bikelane[sdf_bikelane['CLASS'] == '1']

# fix bad values
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['before 2010', ' before 2010'], '2010')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['Before 2006','BEFORE 2006'], '2006')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace([' 2014'], '2014')
filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'] = filtered_sdf_bikelane.loc[:, 'YR_OF_CONS'].replace(['2007 (1A) 2008 (1B)'], '2008')

# drop UNKNOWN values
filtered_sdf_bikelane = filtered_sdf_bikelane[filtered_sdf_bikelane['YR_OF_CONS'] != 'UNKNOWN']

# convert to numeric
filtered_sdf_bikelane['YR_OF_CONS'] = pd.to_numeric(filtered_sdf_bikelane['YR_OF_CONS'])

# create year column
filtered_sdf_bikelane['Year'] = filtered_sdf_bikelane['YR_OF_CONS']

# group by year
df = filtered_sdf_bikelane.groupby(['Year']).size().reset_index(name='MILES')

df


,Year,Total
0,2002,3
1,2003,1
2,2004,2
3,2006,39
4,2008,2
5,2010,11
6,2011,1
7,2012,3
8,2013,2
9,2014,5


### Median Household Income (MHI) by community area (to be determined) and disaggregated by remote and non-remote workers

### Housing costs (median home sales price and rental rates, by jurisdiction); include cost per unit and cost per square foot

### Housing tenure (rented full-time, owner-occupied, second home), disaggregated by race, ethnicity, and age

### Percent of workers who commute into the basin, origin demographics, distance travelled, difference in travel time by mode

### Transient Occupancy Tax revenue and changes over time

In [ ]:
# TBD still waiting on Josh to get this data created.

### Consistent employment and median wages by sector and overall 

### Access to recreation sites, fresh food, and healthcare for zero-vehicle households

### Firewise communities in the Tahoe basin, coolling centers/heating centers, resources, and emergency infrastructure (medical centers with supplies, fire response)

### Population disaggregated by race and ethnicity, age groups

### Number/share of households with access and functional needs (These can be referred to as vulnerable populations including populations such as persons with disabilities, older adults, children, limited English proficiency, and transportation disadvantages)

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import requests
from datetime import datetime


app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='lake-level-chart'),
    dcc.Interval(
        id='interval-component',
        interval=10 * 1000,  # in milliseconds
        n_intervals=0
    ),
])

def get_usgs_data():
    # USGS site number for Lake Tahoe
    site_number = 10337000
    
    # USGS API URL for Lake Tahoe site
    url = f'https://waterservices.usgs.gov/nwis/iv/?format=json&sites={site_number}&parameterCd=00065'

    # Make a request to the USGS API
    response = requests.get(url)
    data = response.json()

    # Extract relevant data
    time_series_data = data['value']['timeSeries'][0]['values'][0]['value']

    # Create a DataFrame
    df = pd.DataFrame(time_series_data)

    # Convert data types
    df['value'] = pd.to_numeric(df['value'])
    df['dateTime'] = pd.to_datetime(df['dateTime'])
    df['value'] = df['value'] + 6220
    return df

@app.callback(
    Output('lake-level-chart', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_chart(n):
    df = get_usgs_data()

    # Create a line chart using Plotly Express
    fig = px.line(df, x='dateTime', y='value', title='Lake Tahoe Water Level')
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='Water Level (ft)')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [1]:
# setup
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureLayer

In [2]:
# setup connection to GIS Portal, this can be and empty GIS() function with a public service
gis = GIS()

In [3]:
# get feature service as a Spatially Enabled Dataframe
service_url = "https://maps.trpa.org/server/rest/services/LTinfo_Climate_Resilience_Dashboard/MapServer/128"
feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a Spatially Enabled Dataframe, this brings the SHAPE with it and can be saved back as a spatial file
sdfParcels = query_result.sdf

In [4]:
sdfParcels

,OBJECTID,variable_code,variable_name,value,Geography,year_sample,dataset,sample_level,Category
0,1,B03002_012E,Total population: Hispanic or Latino,11423,Basin,2020,acs/acs5,block group,Race and Ethnicity
1,2,B03002_012E,Total population: Hispanic or Latino,12151,Basin,2021,acs/acs5,block group,Race and Ethnicity
2,3,DP1_0093C,Total population: Hispanic or Latino,12027,Basin,2020,dec/dp,tract,Race and Ethnicity
3,4,P004003,Total population: Hispanic or Latino,12203,Basin,2010,dec/sf1,tract,Race and Ethnicity
4,5,P008010,Total population: Hispanic or Latino,11035,Basin,2000,dec/sf1,tract,Race and Ethnicity
...,...,...,...,...,...,...,...,...,...
454,455,B25003B_001E,Total: Black Or African American Alone Househo...,92,South Lake,2021,acs/acs5,tract,Tenure by Race
455,456,B25003E_001E,Total: Native Hawaiian And Other Pacific Islan...,8,South Lake,2021,acs/acs5,tract,Tenure by Race
456,457,B25003F_001E,Total: Some Other Race Alone Householder,680,South Lake,2021,acs/acs5,tract,Tenure by Race
457,458,B25003G_001E,Total: Two Or More Races Householder,1006,South Lake,2021,acs/acs5,tract,Tenure by Race


In [46]:
# here's an example of a function to get web service data as a regular Pandas dataframe
def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    # return data frame
    return all_data

In [47]:
feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
feature_list = query_result.features
all_data = pd.DataFrame([feature.attributes for feature in feature_list])
all_data

,OBJECTID,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,NETWORK,GlobalID,created_user,created_date,last_edited_user,last_edited_date
0,1,12428,7-11/Regional Park,39.326248,-120.177104,TART,{B8A5B0A9-9283-4BEA-80F2-CB411AB9DF75},None,None,None,None
1,2,12430,Donner Pass Rd at Donner Memorial State Park,39.32481989,-120.2327303,TART,{12CCE163-5541-4990-99E5-ABCBF6F26FBF},None,None,None,None
2,3,13593,"Ritz Carlton, Highlands Court",39.267333,-120.12784,TART,{EF7721AE-8953-4C77-9F50-2F2FF2CC3673},None,None,None,None
3,4,2120,Country Club Dr at Incline Way,39.24092945,-119.94032,TART,{FAE6196B-FF73-415F-9DE6-EBBF343B5B65},None,None,None,None
4,5,2121,Country Club Dr at Mill Creek,39.24247914,-119.938123,TART,{EDE4C128-052A-4FEB-A0DD-AB72A23CE51B},None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
351,355,9854,Donner Pass Rd at Highway Rd,39.32557709,-120.2222231,TART,{6CA00A77-BB16-4E55-BBD4-3B5CA713C564},None,None,None,None
352,356,9855,Donner Pass Rd Cold Stream Rd,39.32291464,-120.2275942,TART,{FAE71BE9-B377-46FA-93E1-F356D3660855},None,None,None,None
353,357,9856,Donner Pass Rd at Moraine Rd (Sticks Market),39.32895356,-120.2512487,TART,{0207253D-9B1E-4483-A585-3BB9DCE72D83},None,None,None,None
354,358,9858,Donner Pass Rd at Donner Lake Village Resort,39.32480287,-120.2866209,TART,{5751D5A2-8797-437E-AC1F-19075857FFEB},None,None,None,None


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   IndicatorID            85 non-null     int64  
 1   IndicatorName          85 non-null     object 
 2   IndicatorUnits         85 non-null     object 
 3   ProjectID              85 non-null     int64  
 4   EIPProjectNumber       85 non-null     object 
 5   IndicatorProjectYear   85 non-null     int64  
 6   IndicatorProjectValue  85 non-null     float64
 7   PMSubcategoryName1     85 non-null     object 
 8   PMSubcategoryOption1   85 non-null     object 
 9   PMSubcategoryName2     0 non-null      float64
 10  PMSubcategoryOption2   0 non-null      float64
 11  PMSubcategoryName3     0 non-null      float64
 12  PMSubcategoryOption3   0 non-null      float64
 13  PMSubcategoryName4     0 non-null      float64
 14  PMSubcategoryOption4   0 non-null      float64
dtypes: float

In [17]:
df.PMSubcategoryName3.unique()

array([nan])

: 

In [12]:
# get EIP indicator as dataframe
import pandas as pd
import plotly.express as px

eipSEZ = "https://www.laketahoeinfo.org/WebServices/GetReportedEIPIndicatorProjectAccomplishments/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476/9"

df = pd.read_json(eipSEZ)

sum_df = df.groupby('IndicatorProjectYear')['IndicatorProjectValue'].cumsum().reset_index()

# Plot using Plotly Express
fig = px.bar(sum_df, x='IndicatorProjectYear', y='IndicatorProjectValue', title='Sum of Values by Category')
fig.show()
# fig = px.bar(df, x = "IndicatorProjectValue", y = "IndicatorProjectYear")
# fig.show()


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['index', 'IndicatorProjectValue'] but received: IndicatorProjectYear

In [8]:
df 

,IndicatorID,IndicatorName,IndicatorUnits,ProjectID,EIPProjectNumber,IndicatorProjectYear,IndicatorProjectValue,PMSubcategoryName1,PMSubcategoryOption1,PMSubcategoryName2,PMSubcategoryOption2,PMSubcategoryName3,PMSubcategoryOption3,PMSubcategoryName4,PMSubcategoryOption4
0,9,Acres of SEZ Restored or Enhanced,acres,64,01.01.01.0030,2020,1.12,Action Performed,Restored,NaN,NaN,NaN,NaN,NaN,NaN
1,9,Acres of SEZ Restored or Enhanced,acres,154,01.01.01.0045,2015,0.50,Action Performed,Enhanced,NaN,NaN,NaN,NaN,NaN,NaN
2,9,Acres of SEZ Restored or Enhanced,acres,214,01.01.01.0056,2015,0.25,Action Performed,Enhanced,NaN,NaN,NaN,NaN,NaN,NaN
3,9,Acres of SEZ Restored or Enhanced,acres,214,01.01.01.0056,2016,2.00,Action Performed,Enhanced,NaN,NaN,NaN,NaN,NaN,NaN
4,9,Acres of SEZ Restored or Enhanced,acres,229,01.01.01.0067,2020,0.20,Action Performed,Restored,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,9,Acres of SEZ Restored or Enhanced,acres,1645,02.02.01.0015,2015,10.00,Action Performed,Enhanced,NaN,NaN,NaN,NaN,NaN,NaN
81,9,Acres of SEZ Restored or Enhanced,acres,1645,02.02.01.0015,2017,10.00,Action Performed,Enhanced,NaN,NaN,NaN,NaN,NaN,NaN
82,9,Acres of SEZ Restored or Enhanced,acres,1347,03.01.01.0005,2019,0.20,Action Performed,Enhanced,NaN,NaN,NaN,NaN,NaN,NaN
83,9,Acres of SEZ Restored or Enhanced,acres,1474,03.01.02.0048,2014,0.35,Action Performed,Restored,NaN,NaN,NaN,NaN,NaN,NaN
